# Digit Recognizer
#### Learn computer vision fundamentals with the famous MNIST data
Link: https://www.kaggle.com/competitions/digit-recognizer/overview

Name: Tatiana Prado Santos

### 1. Setup and Imports

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
import keras_tuner
from keras_tuner import Hyperband, Objective
import matplotlib.pyplot as plt

# Print versions for clarity and debugging
print("Keras Tuner version:", keras_tuner.__version__)
print("TensorFlow version:", tf.__version__)

2024-06-03 19:38:09.300755: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Keras Tuner version: 1.4.7
TensorFlow version: 2.14.0


### 2. Data Loading and Preprocessing

In [2]:
# Load the datasets
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
sample_submission = pd.read_csv('sample_submission.csv')

x_train = train.drop('label', axis=1).values.reshape(-1, 28, 28, 1) / 255.0
y_train = to_categorical(train['label'], num_classes=10)
x_test = test.values.reshape(-1, 28, 28, 1) / 255.0

### 3. Model Definition

In [3]:
def build_model(hp):
    model = Sequential([
        Conv2D(
            filters=hp.Int('conv_1_filters', min_value=32, max_value=64, step=16),
            kernel_size=hp.Choice('conv_1_kernel_size', values=[3, 5]),
            activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D(pool_size=2),
        Conv2D(
            filters=hp.Int('conv_2_filters', min_value=32, max_value=64, step=16),
            kernel_size=hp.Choice('conv_2_kernel_size', values=[3, 5]),
            activation='relu'),
        MaxPooling2D(pool_size=2),
        Flatten(),
        Dense(
            units=hp.Int('dense_units', min_value=32, max_value=64, step=16), activation='relu'),
        Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-3, 1e-4])),
        loss='categorical_crossentropy', metrics=['accuracy'])
    return model

### 4. Hyperparameter Tuning

In [4]:
tuner = Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=10,
    directory='output',
    project_name='mnist_hyperband'
)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)
tuner.search(x_train, y_train, epochs=10, validation_split=0.2, callbacks=[early_stopping], batch_size=64)

Trial 30 Complete [00h 02m 39s]
val_accuracy: 0.9815475940704346

Best val_accuracy So Far: 0.991428554058075
Total elapsed time: 00h 59m 11s


### 5. Model Training and Evaluation

In [5]:
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
model = tuner.hypermodel.build(best_hps)
history = model.fit(x_train, y_train, epochs=50, validation_split=0.2, batch_size=64)

Epoch 1/50
525/525 [==============================] - 35s 66ms/step - loss: 0.1972 - accuracy: 0.9384 - val_loss: 0.0576 - val_accuracy: 0.9818
Epoch 2/50
525/525 [==============================] - 32s 61ms/step - loss: 0.0544 - accuracy: 0.9835 - val_loss: 0.0555 - val_accuracy: 0.9836
Epoch 3/50
525/525 [==============================] - 33s 64ms/step - loss: 0.0358 - accuracy: 0.9885 - val_loss: 0.0435 - val_accuracy: 0.9869
Epoch 4/50
525/525 [==============================] - 35s 66ms/step - loss: 0.0254 - accuracy: 0.9916 - val_loss: 0.0461 - val_accuracy: 0.9876
Epoch 5/50
525/525 [==============================] - 34s 64ms/step - loss: 0.0186 - accuracy: 0.9936 - val_loss: 0.0442 - val_accuracy: 0.9862
Epoch 6/50
525/525 [==============================] - 34s 65ms/step - loss: 0.0159 - accuracy: 0.9948 - val_loss: 0.0431 - val_accuracy: 0.9885
Epoch 7/50
525/525 [==============================] - 33s 63ms/step - loss: 0.0120 - accuracy: 0.9959 - val_loss: 0.0412 - val_accuracy:

### 6. Model Saving

In [6]:
model.save('mnist_model.h5')
print("Model saved successfully to 'mnist_model.h5'.")
model.save('mnist_model_savedmodel', save_format='tf')
print("Model saved successfully in TensorFlow SavedModel format.")

/opt/anaconda3/lib/python3.9/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model saved successfully to 'mnist_model.h5'.
INFO:tensorflow:Assets written to: mnist_model_savedmodel/assets


INFO:tensorflow:Assets written to: mnist_model_savedmodel/assets


Model saved successfully in TensorFlow SavedModel format.
